In [1]:
%pip install pyspark
%pip install seaborn
%pip install plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [76]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
%matplotlib inline
import warnings

In [77]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HadoopDataRetrieval").enableHiveSupport().getOrCreate()


In [78]:
hdfs_path = "hdfs://localhost:9500/data/country_wise.csv"

# Example for reading a CSV file
df = spark.read.csv(hdfs_path, header=True, inferSchema=True, samplingRatio=0.1)


In [79]:
# Show the first few rows of the DataFrame
df.show()

+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|     Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|
+-------------------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+
|        Afghanistan|    36263|  1269|    25198|  9796|      106|        10|           18|               3.5|                69.49|                  5.04|              35526|          737|             2.07|Eastern Mediterra...|
|            Albania|     4880|   144|     2745|  1991|      117|         6|           6

In [80]:
# Now, read the full dataset
full_df = spark.read.option("header", "true").csv(hdfs_path, header=True, inferSchema=True, samplingRatio=1)

# Save the full DataFrame as a CSV file
output_path = "dataset"
full_df.write.mode("overwrite").csv(output_path, header=True)

In [81]:
def read_data(path,filename):
    return pd.read_csv(path+'/'+filename)

In [82]:
# Access dataset
path = r'C:\Users\lukas\pyspark'
country_wise = read_data(path, 'country_wise.csv')
# Print the first few rows of the data
country_wise.head()

,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,36263,1269,25198,9796,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,Albania,4880,144,2745,1991,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,Algeria,27973,1163,18837,7973,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,Andorra,907,52,803,52,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,Angola,950,41,242,667,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


In [83]:
#What is the trend of confirmed deaths, recovered cases and active cases?

In [84]:
country_wise = read_data(path, 'day-wise-country.csv')
# Print the first few rows of the data
country_wise.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa


In [85]:
day_wise_world = read_data(path, 'day-wise-worldwide.csv')
# Print the first few rows of the data
day_wise_world.head()

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,22/01/2020,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,23/01/2020,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,24/01/2020,941,26,36,879,287,8,6,2.76,3.83,72.22,9
3,25/01/2020,1434,42,39,1353,493,16,3,2.93,2.72,107.69,11
4,26/01/2020,2118,56,52,2010,684,14,13,2.64,2.46,107.69,13
